In [1]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!(cd squad-question-generation/; git pull)
!(cd crf_target_answer/; git pull)

Already up to date.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/soerenetler/crf_target_answer
   7976d56..e5e2c58  main       -> origin/main
Updating 7976d56..e5e2c58
Fast-forward
 requirements.txt | 1 +
 1 file changed, 1 insertion(+)


In [6]:
!pip install -r crf_target_answer/requirements.txt --quiet

     |████████████████████████████████| 112kB 8.2MB/s 
ERROR: eli5 0.11.0 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.


In [8]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2


In [5]:
import joblib
import eli5
import pandas as pd
import ast
import spacy
from sklearn_crfsuite import metrics
from crf_target_answer.model.evaluation_utils import visualize_transitions, print_annotated_text, crf_roc_curve

In [27]:
import json

data_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"
dataset = "quac"
NO_SAMPLES = 10000

with open(data_folder + dataset+'/crf/'+str(NO_SAMPLES)+'_train_crf.txt') as f:
  data_json = json.load(f)
  train_feature = data_json["x"]
  y_train = data_json["y"]

In [30]:
print(train_feature[0])

[{'bias': 1.0, 'AbsolutePosition': 0, 'RelativePosition': 0.0, 'QuatilePosition': 0, '-1 BOS': True, '0:word.pos_': 'NOUN', '0:word.tag_': 'NN', '0:word.dep_': 'compound', '0:word.ent_type': '', '0:word.ent_iob_': 'O', '0:word.lemma': 'film', '0:srl': 'O', '0:srl_verb': 'has', '0:word.is_alpha()': True, '0:word.is_ascii()': True, '0:word.is_digit()': False, '0:word.is_lower()': False, '0:word.is_upper()': False, '0:word.is_title()': True, '0:word.is_punct': False, '0:word.is_left_punct': False, '0:word.is_right_punct': False, '0:word.is_space': False, '0:word.is_bracket': False, '0:word.is_quote': False, '0:word.is_currency': False, '0:word.like_url': False, '0:word.like_num': False, '0:word.like_email': False, '0:word.is_oov': False, '0:word.is_stop': False, '0:word.shape': 'Xxxx', '0:word.prob': -12.764961242675781, '0:word.unigram_proba': 0, '0:word.bigram_proba': 0, '0:word.trigram_proba': 0, '1:word.pos_': 'NOUN', '1:word.tag_': 'NN', '1:word.dep_': 'compound', '1:word.ent_type': 

In [ ]:
def flatten(list_of_lists):
    total_list = []
    for list_1 in list_of_lists:
        total_list += list_1
    return total_list

In [50]:
feature_name = '0:word.is_oov'
value_IO_tuple = list(zip([x.get(feature_name, None) for x in flatten(train_feature)], flatten(y_train)))

In [51]:
import plotly.express as px
px.histogram(x=[x[0] for x in value_IO_tuple],
             facet_row=[x[1] for x in value_IO_tuple],
             title=feature_name, 
             histnorm='percent')

In [ ]:
nlp = spacy.load('en_core_web_md')
sentence = nlp("This is a sentence with sadsad words")
print([token.is_oov for token in sentence])
print([token.prob for token in sentence])

[False, False, False, False, False, True, False]
[-6.78391695022583, -4.457748889923096, -3.92978835105896, -10.146957397460938, -5.243249893188477, -20.0, -8.923008918762207]


In [ ]:
#!python -m spacy download en_core_web_md